# GitHub Copilot + LiteLLM POC

This notebook demonstrates:
1. **LiteLLM GitHub Copilot Integration** - Authentication & Usage
2. **OpenAI Agents Tracing** with non-OpenAI models (GitHub Copilot via LiteLLM)
3. **set_tracing_export_api_key** configuration for trace export

## Prerequisites
- GitHub Copilot subscription/access
- OpenAI API key (for tracing to OpenAI dashboard)
- LiteLLM installed: `pip install litellm openai`


## Section 1: Setup & Environment Configuration

In [1]:
import os
from pathlib import Path
from dotenv import dotenv_values

# 找到 .env 檔案
env_file = Path.cwd().parent.parent / ".env"
if not env_file.exists():
    env_file = Path.home() / "Documents/workspace/CasualTrader/backend/.env"

# 讀取並設定環境變數
if env_file.exists():
    env_vars = dotenv_values(env_file)
    for key, value in env_vars.items():
        os.environ[key] = value.strip('"').strip("'") if value else ""

# 取得 API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
GITHUB_PERSONAL_ACCESS_TOKEN = os.getenv("GITHUB_PERSONAL_ACCESS_TOKEN", "")

print(f"✓ .env 已載入: {env_file}")
print(f"✓ OPENAI_API_KEY: {OPENAI_API_KEY[:20]}...{OPENAI_API_KEY[-20:]}" if OPENAI_API_KEY else "✗ 未設定")
print(f"✓ GITHUB_PERSONAL_ACCESS_TOKEN: {GITHUB_PERSONAL_ACCESS_TOKEN[:20]}...{GITHUB_PERSONAL_ACCESS_TOKEN[-20:]}" if GITHUB_PERSONAL_ACCESS_TOKEN else "✗ 未設定")

✓ .env 已載入: /Users/sacahan/Documents/workspace/CasualTrader/backend/.env
✓ OPENAI_API_KEY: sk-proj-GIGRm9otjijc...azDgPPVzzYFZUAuyz0QA
✓ GITHUB_PERSONAL_ACCESS_TOKEN: ghp_HJYjAlbBoxrUSnE0...9nt7LU6xZnoIjQ2gt793


## Section 2: LiteLLM GitHub Copilot - Authentication & Basic Usage

In [3]:
from litellm import completion, acompletion
import asyncio

# Test 1: Non-streaming completion with GitHub Copilot
# Note: On first run, LiteLLM will prompt with OAuth device code

def test_github_copilot_basic():
    """
    Test basic GitHub Copilot completion via LiteLLM.
    
    First execution triggers OAuth device flow:
    - LiteLLM displays a device code and verification URL
    - Visit the URL and enter the code
    - Authenticate with your GitHub account
    - OAuth tokens are cached for future use
    """

    model = "github_copilot/gemini-2.5-pro"

    print("\n=== Test 1: Basic Completion (Non-Streaming) ===")
    print(f"Model: {model}")
    print("Status: Awaiting response...\n")
    
    try:
        response = completion(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": "Write a Python function to calculate the Fibonacci sequence up to n numbers, just python code without any explanation",
                }
            ],
            # temperature=0.7,
            max_tokens=500,
            extra_headers={
                "editor-version": "vscode/1.85.1",
                "Copilot-Integration-Id": "vscode-chat",
            },
        )
        
        # print("Response received:")
        print("-" * 50)
        print(response["choices"][0]["message"]["content"])
        print("-" * 50)
        # print(f"\nUsage: {response.get('usage', 'N/A')}")
        
    except Exception as e:
        print(f"Error: {type(e).__name__}")
        print(f"Message: {str(e)}")
        print("\nNote: This is expected if:")
        print("  1. OAuth authentication is needed (first run)")
        print("  2. GitHub Copilot subscription is not active")
        print("  3. Network connectivity issue")

# Run the test
test_github_copilot_basic()


=== Test 1: Basic Completion (Non-Streaming) ===
Model: github_copilot/gemini-2.5-pro
Status: Awaiting response...

--------------------------------------------------
```python
def fibonacci(n):
    """
    Calculates the Fibonacci sequence up to n numbers.

    Args:
        n: The number of Fibonacci numbers to generate.

    Returns:
        A list containing the first n Fibonacci numbers.
    """
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    
    sequence = [0, 1]
    while len(sequence) < n:
        next_value = sequence[-1] + sequence[-2]
        sequence.append(next_value)
        
    return sequence

# Example of how to use the function:
# if __name__ == '__main__':
#     num_terms = 10
#     fib_sequence = fibonacci(num_terms)
#     print(f"The Fibonacci sequence up to {num_terms} numbers is: {fib_sequence}")
#     # Expected output: The Fibonacci sequence up to 10 numbers is: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
```
----------------------------------

## Section 3: LiteLLM GitHub Copilot - Streaming

In [ ]:
def test_github_copilot_streaming():
    """
    Test streaming responses from GitHub Copilot via LiteLLM.
    
    Streaming is useful for real-time feedback in UI applications.
    """

    model = "github_copilot/grok-code-fast-1"

    print("\n=== Test 2: Streaming Completion ===")
    print(f"Model: {model}")
    print("Status: Streaming response...\n")
    
    try:
        stream = completion(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": "Explain async/await in Python with a simple example"
                }
            ],
            stream=True,
            # temperature=0.8,
            max_tokens=300,
            extra_headers={
                "editor-version": "vscode/1.85.1",
                "Copilot-Integration-Id": "vscode-chat"
            }
        )
        
        print("Streaming response:")
        print("-" * 50)
        
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        
        print("\n" + "-" * 50)
        print(f"\nTotal characters received: {len(full_response)}")
        
    except Exception as e:
        print(f"Error: {type(e).__name__}: {str(e)}")

# Run the test
test_github_copilot_streaming()

## Section 4: OpenAI Agents with GitHub Copilot + Tracing Setup

In [ ]:
# Import OpenAI Agents SDK components
try:
    from openai import OpenAI
    from agents import set_tracing_export_api_key, Agent, tool, RunConfig
    print("✓ OpenAI Agents SDK imported successfully")
except ImportError as e:
    print(f"⚠ OpenAI Agents SDK not installed: {e}")
    print("Install with: pip install openai[agents]")
    print("Note: This section requires the latest openai package with agents support")

## Section 5: Configure Tracing for Non-OpenAI Models

When using GitHub Copilot (non-OpenAI model) with OpenAI Agents tracing:
- The LLM backend is GitHub Copilot (via LiteLLM)
- Traces are exported to OpenAI's Traces Dashboard
- Use `set_tracing_export_api_key()` to set the OpenAI API key for trace export only

In [ ]:
# Configuration for tracing with non-OpenAI models

# Step 1: Set the OpenAI API key for trace export
# This is separate from the GitHub Copilot authentication

def setup_tracing_for_non_openai_model():
    """
    Configure tracing for non-OpenAI models (GitHub Copilot via LiteLLM).
    
    Key Points:
    1. LLM requests go to GitHub Copilot (via LiteLLM)
    2. Trace events are exported to OpenAI's Traces Dashboard
    3. set_tracing_export_api_key() decouples trace export from LLM API key
    
    Note:
    - set_tracing_export_api_key is GLOBAL (affects all agents)
    - For per-agent or per-project tracing, use set_trace_processors()
    """
    try:
        # Set the trace export API key (for OpenAI Traces Dashboard)
        set_tracing_export_api_key(OPENAI_API_KEY)
        print("✓ Tracing configured for non-OpenAI models")
        print(f"  - Trace export API key set")
        print(f"  - Traces will be exported to OpenAI Traces Dashboard")
        print(f"  - LLM backend: GitHub Copilot (via LiteLLM)")
        
    except Exception as e:
        print(f"⚠ Tracing setup failed: {e}")
        print("  - Continue with tracing disabled or use environment variables")

setup_tracing_for_non_openai_model()

## Section 6: Example - Using GitHub Copilot with Agents SDK

In [ ]:
# Example configuration for using GitHub Copilot with OpenAI Agents

from agents import Agent, Runner, function_tool, ModelSettings
from agents.extensions.models.litellm_model import LitellmModel
import json

model = "github_copilot/gpt-5-mini"

model_settings = ModelSettings(
    include_usage=True,
    extra_headers={
        "editor-version": "vscode/1.85.1",
        "Copilot-Integration-Id": "vscode-chat",
    }
)


@function_tool
def get_weather(city: str):
    print(f"[debug] getting weather for {city}")
    return f"The weather in {city} is sunny."

agent = Agent(
    name="Assistant",
    instructions="You only respond in haikus.",
    model=LitellmModel(model=model, api_key=GITHUB_PERSONAL_ACCESS_TOKEN),
    tools=[get_weather],
    model_settings=model_settings,
)

result = await Runner.run(agent, "What's the weather in Tokyo?")
print(json.dumps(result.final_output, indent=2))

print("\n=== GitHub Copilot + Agents Configuration ===")

## Section 7: Simple Agent Testing

這個部分進行基本的 Agent 功能測試：
- 簡單工具測試 (天氣查詢、數學計算)
- 基本的 Agent 交互

In [2]:
# Simple Agent Testing

import json
from agents import Agent, Runner, function_tool, ModelSettings
from agents.extensions.models.litellm_model import LitellmModel

# Configuration
model = "github_copilot/gpt-5"
GITHUB_PERSONAL_ACCESS_TOKEN = os.getenv("GITHUB_COPILOT_TOKEN", "")

model_settings = ModelSettings(
    include_usage=True,
    extra_headers={
        "editor-version": "vscode/1.85.1",
        "Copilot-Integration-Id": "vscode-chat",
    }
)

print(f"Model: {model}")

# Simple Tools
@function_tool
def get_weather(city: str) -> str:
    """Get weather for a city"""
    return f"The weather in {city} is sunny, 22°C"

@function_tool
def calculate_sum(a: float, b: float) -> float:
    """Add two numbers"""
    return a + b

# Helper function to run a single, isolated test
async def run_test(test_name, query):
    """Helper to run a test with a fresh agent and print the output."""
    print(f"--- {test_name} ---")
    
    # Create a new agent for each test to ensure a clean state
    agent = Agent(
        name="Assistant",
        instructions="You are a helpful assistant. Keep responses concise.",
        model=LitellmModel(model=model, api_key=GITHUB_PERSONAL_ACCESS_TOKEN),
        tools=[get_weather, calculate_sum],
        model_settings=model_settings,
    )
    
    try:
        result = await Runner.run(agent, query)
        print(result.final_output)
    except Exception as e:
        print(f"Error: {type(e).__name__}: {str(e)}")
    print("-" * 40)

# Basic Test
async def test_simple_agent():
    """Test basic agent functionality with isolated runs."""
    print("\n=== 簡單 Agent 測試 ===\n")
    
    await run_test("測試 1: 天氣查詢", "What's the weather in Tokyo?")
    # await run_test("測試 2: 數學計算", "What is 15 + 25?")

# Run the test
await test_simple_agent()

Model: github_copilot/gpt-5

=== 簡單 Agent 測試 ===

--- 測試 1: 天氣查詢 ---
Tokyo weather: Sunny, 22°C.
----------------------------------------
Tokyo weather: Sunny, 22°C.
----------------------------------------
